In [72]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import os

In [8]:
data_dir = 'P_FINE_DATA'
def load_split_train_test(datadir, valid_size = .2):
    train_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                       transforms.ToTensor(),
                                       ])
    test_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.ToTensor(),
                                      ])
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=64)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=64)
    return trainloader, testloader
trainloader, testloader = load_split_train_test(data_dir, .2)
print(trainloader.dataset.classes)

['c1@1', 'c1@2', 'c1@3', 'c1@4', 'c1@5', 'c1@6', 'c1@7', 'c2@1', 'c2@10', 'c2@11', 'c2@2', 'c2@3', 'c2@4', 'c2@5', 'c2@6', 'c2@7', 'c2@8', 'c2@9', 'c3@1', 'c3@2', 'c3@3', 'c3@4', 'c3@5', 'c3@6', 'c3@7', 'c3@8', 'c4@1', 'c4@2', 'c4@3', 'c4@4', 'c4@5', 'c5@1', 'c5@2', 'c5@3', 'c5@4', 'c5@5']


# COARSE_MODEL --------------------------------------

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_coarse = torch.load('coarse_model.pth')
model_coarse.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

# ----------------------------------------------------------

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
model = models.resnet50(pretrained=True)

In [11]:
for param in model.parameters():
    param.requires_grad = False
    
model.fc = nn.Sequential(nn.Linear(2048, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 36),
                                 nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [49]:
# epochs = 10
# steps = 0
# running_loss = 0
# print_every = 10
# train_losses, test_losses = [], []
# for epoch in range(epochs):
#     for inputs, labels in trainloader:
#         steps += 1
#         inputs, labels = inputs.to(device), labels.to(device)
#         logps2 = model_coarse.forward(inputs)
#         a = logps2.cpu().detach().numpy()
#         optimizer.zero_grad()
#         logps3 = model.forward(inputs)
#         logps = torch.zeros([len(logps2), 36], dtype=torch.float64)
#         #logps3 = logps
#         for i in range(7):
#             logps[:,i] = logps3[:,i].clone() + logps2[:,0]
#         for i in range(7,18):
#             logps[:,i] = logps3[:,i].clone() + logps2[:,1]
#         for i in range(18,26):
#             logps[:,i] = logps3[:,i].clone() + logps2[:,2]
#         for i in range(26,31):
#             logps[:,i] = logps3[:,i].clone() + logps2[:,3]
#         for i in range(31,36):
#             logps[:,i] = logps3[:,i].clone() + logps2[:,4]
        
#         logps4 = torch.zeros([len(logps2), 36], dtype=torch.float64)
#         logps4 = logps4 + logps
#         loss = criterion(logps4, labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
        
#         if steps % print_every == 0:
#             test_loss = 0
#             accuracy = 0
#             model.eval()
#             with torch.no_grad():
#                 for inputs, labels in testloader:
#                     inputs, labels = inputs.to(device),labels.to(device)
#                     logps = model.forward(inputs)
#                     batch_loss = criterion(logps, labels)
#                     test_loss += batch_loss.item()
                    
#                     ps = torch.exp(logps)
#                     top_p, top_class = ps.topk(1, dim=1)
#                     equals = top_class == labels.view(*top_class.shape)
#                     accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
#             train_losses.append(running_loss/len(trainloader))
#             test_losses.append(test_loss/len(testloader))                    
#             print(f"Epoch {epoch+1}/{epochs}.. "
#                   f"Train loss: {running_loss/print_every:.3f}.. "
#                   f"Test loss: {test_loss/len(testloader):.3f}.. "
#                   f"Test accuracy: {accuracy/len(testloader):.3f}")
#             running_loss = 0
#             model.train()
# torch.save(model, 'fine_model.pth')

Epoch 1/10.. Train loss: 0.722.. Test loss: 0.641.. Test accuracy: 0.799
Epoch 1/10.. Train loss: 0.788.. Test loss: 0.565.. Test accuracy: 0.814
Epoch 1/10.. Train loss: 0.625.. Test loss: 0.587.. Test accuracy: 0.795
Epoch 2/10.. Train loss: 0.787.. Test loss: 0.645.. Test accuracy: 0.782
Epoch 2/10.. Train loss: 0.644.. Test loss: 0.529.. Test accuracy: 0.812
Epoch 2/10.. Train loss: 0.562.. Test loss: 0.501.. Test accuracy: 0.822
Epoch 3/10.. Train loss: 0.542.. Test loss: 0.526.. Test accuracy: 0.819
Epoch 3/10.. Train loss: 0.569.. Test loss: 0.524.. Test accuracy: 0.833
Epoch 3/10.. Train loss: 0.569.. Test loss: 0.496.. Test accuracy: 0.820
Epoch 4/10.. Train loss: 0.534.. Test loss: 0.524.. Test accuracy: 0.818
Epoch 4/10.. Train loss: 0.583.. Test loss: 0.486.. Test accuracy: 0.829
Epoch 4/10.. Train loss: 0.486.. Test loss: 0.515.. Test accuracy: 0.816
Epoch 5/10.. Train loss: 0.510.. Test loss: 0.424.. Test accuracy: 0.853
Epoch 5/10.. Train loss: 0.433.. Test loss: 0.467..

In [50]:
# epochs = 10
# steps = 0
# running_loss = 0
# print_every = 10
# train_losses, test_losses = [], []
# for epoch in range(epochs):
#     for inputs, labels in trainloader:
#         steps += 1
#         inputs, labels = inputs.to(device), labels.to(device)
#         logps2 = model_coarse.forward(inputs)
#         a = logps2.cpu().detach().numpy()
#         optimizer.zero_grad()
#         logps3 = model.forward(inputs)
#         logps = torch.zeros([len(logps2), 36], dtype=torch.float64)
#         #logps3 = logps
#         for i in range(7):
#             logps[:,i] = logps3[:,i].clone() + logps2[:,0]
#         for i in range(7,18):
#             logps[:,i] = logps3[:,i].clone() + logps2[:,1]
#         for i in range(18,26):
#             logps[:,i] = logps3[:,i].clone() + logps2[:,2]
#         for i in range(26,31):
#             logps[:,i] = logps3[:,i].clone() + logps2[:,3]
#         for i in range(31,36):
#             logps[:,i] = logps3[:,i].clone() + logps2[:,4]
        
#         logps4 = torch.zeros([len(logps2), 36], dtype=torch.float64)
#         logps4 = logps4 + logps
#         loss = criterion(logps4, labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
        
#         if steps % print_every == 0:
#             test_loss = 0
#             accuracy = 0
#             model.eval()
#             with torch.no_grad():
#                 for inputs, labels in testloader:
#                     inputs, labels = inputs.to(device),labels.to(device)
#                     logps = model.forward(inputs)
#                     batch_loss = criterion(logps, labels)
#                     test_loss += batch_loss.item()
                    
#                     ps = torch.exp(logps)
#                     top_p, top_class = ps.topk(1, dim=1)
#                     equals = top_class == labels.view(*top_class.shape)
#                     accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
#             train_losses.append(running_loss/len(trainloader))
#             test_losses.append(test_loss/len(testloader))                    
#             print(f"Epoch {epoch+1}/{epochs}.. "
#                   f"Train loss: {running_loss/print_every:.3f}.. "
#                   f"Test loss: {test_loss/len(testloader):.3f}.. "
#                   f"Test accuracy: {accuracy/len(testloader):.3f}")
#             running_loss = 0
#             model.train()
# torch.save(model, 'fine_model.pth')

Epoch 1/10.. Train loss: 0.417.. Test loss: 0.446.. Test accuracy: 0.850
Epoch 1/10.. Train loss: 0.376.. Test loss: 0.434.. Test accuracy: 0.856
Epoch 1/10.. Train loss: 0.402.. Test loss: 0.423.. Test accuracy: 0.844
Epoch 2/10.. Train loss: 0.394.. Test loss: 0.434.. Test accuracy: 0.846
Epoch 2/10.. Train loss: 0.431.. Test loss: 0.453.. Test accuracy: 0.854
Epoch 2/10.. Train loss: 0.403.. Test loss: 0.395.. Test accuracy: 0.869
Epoch 3/10.. Train loss: 0.464.. Test loss: 0.400.. Test accuracy: 0.853
Epoch 3/10.. Train loss: 0.403.. Test loss: 0.512.. Test accuracy: 0.821
Epoch 3/10.. Train loss: 0.400.. Test loss: 0.400.. Test accuracy: 0.858
Epoch 4/10.. Train loss: 0.510.. Test loss: 0.381.. Test accuracy: 0.861
Epoch 4/10.. Train loss: 0.393.. Test loss: 0.393.. Test accuracy: 0.867
Epoch 4/10.. Train loss: 0.388.. Test loss: 0.456.. Test accuracy: 0.854
Epoch 5/10.. Train loss: 0.442.. Test loss: 0.432.. Test accuracy: 0.847
Epoch 5/10.. Train loss: 0.388.. Test loss: 0.334..

In [52]:
# optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
# epochs = 5
# steps = 0
# running_loss = 0
# print_every = 10
# train_losses, test_losses = [], []
# for epoch in range(epochs):
#     for inputs, labels in trainloader:
#         steps += 1
#         inputs, labels = inputs.to(device), labels.to(device)
#         logps2 = model_coarse.forward(inputs)
#         a = logps2.cpu().detach().numpy()
#         optimizer.zero_grad()
#         logps3 = model.forward(inputs)
#         logps = torch.zeros([len(logps2), 36], dtype=torch.float64)
#         #logps3 = logps
#         for i in range(7):
#             logps[:,i] = logps3[:,i].clone() + logps2[:,0]
#         for i in range(7,18):
#             logps[:,i] = logps3[:,i].clone() + logps2[:,1]
#         for i in range(18,26):
#             logps[:,i] = logps3[:,i].clone() + logps2[:,2]
#         for i in range(26,31):
#             logps[:,i] = logps3[:,i].clone() + logps2[:,3]
#         for i in range(31,36):
#             logps[:,i] = logps3[:,i].clone() + logps2[:,4]
        
#         logps4 = torch.zeros([len(logps2), 36], dtype=torch.float64)
#         logps4 = logps4 + logps
#         loss = criterion(logps4, labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
        
#         if steps % print_every == 0:
#             test_loss = 0
#             accuracy = 0
#             model.eval()
#             with torch.no_grad():
#                 for inputs, labels in testloader:
#                     inputs, labels = inputs.to(device),labels.to(device)
#                     logps = model.forward(inputs)
#                     batch_loss = criterion(logps, labels)
#                     test_loss += batch_loss.item()
                    
#                     ps = torch.exp(logps)
#                     top_p, top_class = ps.topk(1, dim=1)
#                     equals = top_class == labels.view(*top_class.shape)
#                     accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
#             train_losses.append(running_loss/len(trainloader))
#             test_losses.append(test_loss/len(testloader))                    
#             print(f"Epoch {epoch+1}/{epochs}.. "
#                   f"Train loss: {running_loss/print_every:.3f}.. "
#                   f"Test loss: {test_loss/len(testloader):.3f}.. "
#                   f"Test accuracy: {accuracy/len(testloader):.3f}")
#             running_loss = 0
#             model.train()
# torch.save(model, 'fine_model.pth')

Epoch 1/5.. Train loss: 0.642.. Test loss: 0.572.. Test accuracy: 0.798
Epoch 1/5.. Train loss: 0.689.. Test loss: 0.482.. Test accuracy: 0.828
Epoch 1/5.. Train loss: 0.565.. Test loss: 0.485.. Test accuracy: 0.826
Epoch 2/5.. Train loss: 0.606.. Test loss: 0.487.. Test accuracy: 0.836
Epoch 2/5.. Train loss: 0.528.. Test loss: 0.485.. Test accuracy: 0.857
Epoch 2/5.. Train loss: 0.534.. Test loss: 0.385.. Test accuracy: 0.835
Epoch 3/5.. Train loss: 0.480.. Test loss: 0.453.. Test accuracy: 0.840
Epoch 3/5.. Train loss: 0.455.. Test loss: 0.444.. Test accuracy: 0.839
Epoch 3/5.. Train loss: 0.515.. Test loss: 0.429.. Test accuracy: 0.864
Epoch 4/5.. Train loss: 0.511.. Test loss: 0.440.. Test accuracy: 0.827
Epoch 4/5.. Train loss: 0.424.. Test loss: 0.403.. Test accuracy: 0.859
Epoch 4/5.. Train loss: 0.432.. Test loss: 0.420.. Test accuracy: 0.858
Epoch 5/5.. Train loss: 0.413.. Test loss: 0.511.. Test accuracy: 0.799
Epoch 5/5.. Train loss: 0.462.. Test loss: 0.495.. Test accuracy

# GENERATE TEST OUTPUT

In [146]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_fine =torch.load('fine_model.pth')
model_fine.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [147]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_coarse =torch.load('coarse_model.pth')
model_coarse.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [148]:
data_dir = 'test_images'
test_transforms = transforms.Compose([transforms.Resize(224),transforms.CenterCrop(224),
                                      transforms.ToTensor()
                                     ])

In [149]:
def predict_coarse_label_image(image):
    image_tensor = test_transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    inp = image_tensor
    inp = inp.to(device)
    output = model_coarse(inp)
    ps = torch.exp(output)
    index = output.data.cpu().numpy().argmax()
    return index,ps

In [150]:
def predict_fine_label_image(image):
    image_tensor = test_transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    inp = image_tensor
    inp = inp.to(device)
    output = model_fine(inp)
    ps = torch.exp(output)
    index = output.data.cpu().numpy().argmax()
    return index,ps

In [169]:
coarse_names = {0:'aircrafts', 1:'birds', 2:'cars', 3:'dogs', 4:'flowers' }

In [167]:
fine_names = { 0:'aircrafts@1', 1:'aircrafts@2', 2:'aircrafts@3', 3:'aircrafts@4', 4:'aircrafts@5', 5:'aircrafts@6', 6:'aircrafts@7', 
              7:'birds@1', 8:'birds@10', 9:'birds@11', 10:'birds@2', 11:'birds@3', 12:'birds@4', 13:'birds@5', 
              14:'birds@6', 15:'birds@7', 16:'birds@8', 17:'birds@9', 18:'cars@1', 19:'cars@2', 20:'cars@3', 
              21:'cars@4', 22:'cars@5', 23:'cars@6', 24:'cars@7', 25:'cars@8', 26:'dogs@1', 27:'dogs@2', 
              28:'dogs@3', 29:'dogs@4', 30:'dogs@5', 31:'flowers@1', 32:'flowers@2', 33:'flowers@3', 34:'flowers@4',
              35:'flowers@5'}

In [153]:
import torch
from torchvision import datasets

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

# EXAMPLE USAGE:
# instantiate the dataset and dataloader
data_dir = "test_images"
dataset = ImageFolderWithPaths(data_dir) # our custom dataset
dataloader = torch.utils.data.DataLoader(dataset)

# iterate over data


In [162]:
image_names = []
for i in range(len(dataset)):
    image_names.append(dataset[i][2][17:])

In [163]:
images = []
for i in range(len(dataset)):
    images.append(dataset[i][0])

In [170]:
to_pil = transforms.ToPILImage()
to_tensor = transforms.ToTensor()
# images,_ = get_random_images(len(image_names))
# fig=plt.figure(figsize=(10,10))
file = open("output.txt","w") 
for ii in range(len(images)):
    tensor = to_tensor(images[ii])
    image = to_pil(tensor)
    index1,ps = predict_coarse_label_image(image)
    index2,ps1 = predict_fine_label_image(image)
    file.write(image_names[ii]+' '+ coarse_names[index1] + ' ' + fine_names[index2]+'\n')
    #probs.append(ps)
    #sub = fig.add_subplot(1, len(images), ii+1)
#     res = labels[ii]
    #sub.set_title(str(trainloader.dataset.classes[index]) + ":" + str(res))
#     plt.axis('off')
#     plt.imshow(image)
# plt.show()
file.close()